正在加载CIFAR-10数据集...
使用设备: cpu


NameError: name 'ader' is not defined

In [ ]:
# 检查并安装必要的依赖
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.metrics import accuracy_score, classification_report
except ImportError as e:
    print(f"缺少基础包: {e}")
    print("请运行: pip install torch numpy matplotlib scikit-learn")

try:
    from torchvision import datasets, transforms
except ImportError:
    print("正在安装torchvision和pillow...")
    install_package("torchvision")
    install_package("pillow")
    from torchvision import datasets, transforms

# 1. 数据加载与预处理
print("正在加载CIFAR-10数据集...")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

try:
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
except Exception as e:
    print(f"数据下载失败: {e}")
    print("请检查网络连接或手动下载CIFAR-10数据集")

train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# 2. 定义改进的CNN
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ImprovedCNN, self).__init__()
        # 第一个卷积块
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        # 第二个卷积块
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        # 池化层和激活函数
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.5)
        # 全连接层
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# 3. 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

model = ImprovedCNN().to(device)
print(f"模型参数数量: {sum(p.numel() for p in model.parameters()):,}")

# 4. 训练配置
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# 5. 训练循环
epochs = 15
train_losses = []
train_accuracies = []

print("开始训练...")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        if batch_idx % 200 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}')
    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

# 6. 测试与评估
print("开始测试...")
model.eval()
test_loss = 0
correct = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        all_preds.extend(pred.cpu().numpy().flatten())
        all_labels.extend(target.cpu().numpy())
test_loss /= len(test_loader)
test_accuracy = 100. * correct / len(testset)
print(f'测试集平均损失: {test_loss:.4f}')
print(f'测试集准确率: {test_accuracy:.2f}% ({correct}/{len(testset)})')

# 7. 详细分类报告
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
print("\n分类报告:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# 8. 可视化训练过程
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.title('训练损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.subplot(1, 2, 2)
plt.plot(train_accuracies)
plt.title('训练准确率')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.tight_layout()
plt.show()

# 9. 可视化预测结果
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i in range(10):
    img, true_label = testset[i]
    pred_label = all_preds[i]
    img_show = img * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    img_show = torch.clamp(img_show, 0, 1)
    ax = axes[i//5, i%5]
    ax.imshow(np.transpose(img_show.numpy(), (1, 2, 0)))
    color = 'green' if true_label == pred_label else 'red'
    ax.set_title(f"真实: {class_names[true_label]}\n预测: {class_names[pred_label]}", color=color, fontsize=10)
    ax.axis('off')
plt.tight_layout()
plt.show()

print("程序执行完成！")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "e:\avm\ado\envs\py310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\avm\ado\envs\py310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "e:\avm\ado\envs\py310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "e:\avm\ado\envs\py310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "e:\avm\ado\envs\py310\lib\site-packages\ipykernel

正在加载CIFAR-10数据集...
Files already downloaded and verified
Files already downloaded and verified
使用设备: cuda
模型参数数量: 2,168,746
开始训练...


In [ ]:
# 检查并安装必要的依赖
import sys
import subprocess
import time

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, Subset
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.metrics import accuracy_score, classification_report
except ImportError as e:
    print(f"缺少基础包: {e}")
    print("请运行: pip install torch numpy matplotlib scikit-learn")

try:
    from torchvision import datasets, transforms
except ImportError:
    print("正在安装torchvision和pillow...")
    install_package("torchvision")
    install_package("pillow")
    from torchvision import datasets, transforms

# 1. 数据加载与预处理
print("正在加载CIFAR-10数据集...")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

try:
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
except Exception as e:
    print(f"数据下载失败: {e}")
    print("请检查网络连接或手动下载CIFAR-10数据集")

# 使用部分数据 - 训练集使用10%，测试集使用20%
train_size = len(trainset) // 10  # 5000个样本
test_size = len(testset) // 5     # 2000个样本

train_indices = torch.randperm(len(trainset))[:train_size]
test_indices = torch.randperm(len(testset))[:test_size]

train_subset = Subset(trainset, train_indices)
test_subset = Subset(testset, test_indices)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False, num_workers=2)

print(f"使用训练样本: {len(train_subset)}")
print(f"使用测试样本: {len(test_subset)}")

# 2. 定义改进的CNN
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ImprovedCNN, self).__init__()
        # 第一个卷积块
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        # 第二个卷积块
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        # 池化层和激活函数
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.5)
        # 全连接层
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# 3. 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

model = ImprovedCNN().to(device)
print(f"模型参数数量: {sum(p.numel() for p in model.parameters()):,}")

# 4. 训练配置
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 5. 训练循环 - 减少epochs
epochs = 5  # 减少训练轮数
train_losses = []
train_accuracies = []

print("开始训练...")
start_time = time.time()

for epoch in range(epochs):
    epoch_start = time.time()
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        if batch_idx % 50 == 0:  # 更频繁地打印进度
            print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx}, Loss: {loss.item():.4f}')
    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    epoch_time = time.time() - epoch_start
    print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, 时间: {epoch_time:.1f}秒')

total_time = time.time() - start_time
print(f'总训练时间: {total_time/60:.1f}分钟')

# 6. 测试与评估
print("开始测试...")
model.eval()
test_loss = 0
correct = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        all_preds.extend(pred.cpu().numpy().flatten())
        all_labels.extend(target.cpu().numpy())
test_loss /= len(test_loader)
test_accuracy = 100. * correct / len(test_subset)
print(f'测试集平均损失: {test_loss:.4f}')
print(f'测试集准确率: {test_accuracy:.2f}% ({correct}/{len(test_subset)})')

# 7. 详细分类报告
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
print("\n分类报告:")
print(classification_report(all_labels, all_preds, target_names=class_names))

# 8. 可视化训练过程
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses)
plt.title('训练损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.subplot(1, 2, 2)
plt.plot(train_accuracies)
plt.title('训练准确率')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.tight_layout()
plt.show()

# 9. 可视化部分预测结果
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i in range(min(10, len(test_subset))):
    img, true_label = test_subset[i]
    pred_label = all_preds[i]
    img_show = img * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    img_show = torch.clamp(img_show, 0, 1)
    ax = axes[i//5, i%5]
    ax.imshow(np.transpose(img_show.numpy(), (1, 2, 0)))
    color = 'green' if true_label == pred_label else 'red'
    ax.set_title(f"真实: {class_names[true_label]}\n预测: {class_names[pred_label]}", color=color, fontsize=10)
    ax.axis('off')
plt.tight_layout()
plt.show()

print("程序执行完成！")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "E:\avm\ado\envs\py310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "E:\avm\ado\envs\py310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "E:\avm\ado\envs\py310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "E:\avm\ado\envs\py310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "E:\avm\ado\envs\py310\lib\site-packages\ipykernel

正在加载CIFAR-10数据集...
Files already downloaded and verified
Files already downloaded and verified
使用训练样本: 5000
使用测试样本: 2000
使用设备: cuda
模型参数数量: 2,168,746
开始训练...


In [2]:
# 检查并安装必要的依赖
import sys
import subprocess
import time

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, Subset
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.metrics import accuracy_score, classification_report
except ImportError as e:
    print(f"缺少基础包: {e}")
    install_package("torch numpy matplotlib scikit-learn")

try:
    from torchvision import datasets, transforms
except ImportError:
    print("正在安装torchvision和pillow...")
    install_package("torchvision")
    install_package("pillow")
    from torchvision import datasets, transforms

print("开始加载CIFAR-10数据集...")

# 1. 数据预处理 - 简化版本
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 简化的标准化
])

try:
    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    print("数据集下载完成！")
except Exception as e:
    print(f"数据下载失败: {e}")

# 2. 大幅减少数据集大小 - 每类只用100个样本训练，20个样本测试
def create_small_dataset(dataset, samples_per_class=100):
    targets = np.array(dataset.targets)
    indices = []
    for class_id in range(10):
        class_indices = np.where(targets == class_id)[0]
        selected = np.random.choice(class_indices, min(samples_per_class, len(class_indices)), replace=False)
        indices.extend(selected)
    return Subset(dataset, indices)

np.random.seed(42)  # 固定随机种子
small_trainset = create_small_dataset(trainset, samples_per_class=100)  # 1000个训练样本
small_testset = create_small_dataset(testset, samples_per_class=20)     # 200个测试样本

train_loader = DataLoader(small_trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(small_testset, batch_size=32, shuffle=False)

print(f"训练样本数: {len(small_trainset)}")
print(f"测试样本数: {len(small_testset)}")

# 3. 定义轻量级CNN模型
class LightweightCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(LightweightCNN, self).__init__()
        # 简化的卷积层
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

        # 全连接层
        self.fc1 = nn.Linear(32 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# 4. 模型初始化
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

model = LightweightCNN().to(device)
print(f"模型参数数量: {sum(p.numel() for p in model.parameters()):,}")

# 5. 训练配置
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 6. 快速训练 - 只训练3个epoch
epochs = 3
train_losses = []
train_accuracies = []

print("开始快速训练...")
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

training_time = time.time() - start_time
print(f'训练完成！用时: {training_time:.1f}秒')

# 7. 快速测试
print("开始测试...")
model.eval()
correct = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(target.cpu().numpy())

test_accuracy = 100. * correct / len(small_testset)
print(f'测试准确率: {test_accuracy:.2f}% ({correct}/{len(small_testset)})')

# 8. 可视化训练过程
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, 'b-', marker='o')
plt.title('训练损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, 'r-', marker='s')
plt.title('训练准确率')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True)

plt.tight_layout()
plt.show()

# 9. 可视化预测结果
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i in range(10):
    if i >= len(small_testset):
        break

    # 获取测试数据
    img, true_label = small_testset[i]
    pred_label = all_preds[i]

    # 反标准化显示图像
    img_show = img * 0.5 + 0.5
    img_show = torch.clamp(img_show, 0, 1)

    ax = axes[i//5, i%5]
    ax.imshow(np.transpose(img_show.numpy(), (1, 2, 0)))

    # 设置标题颜色
    color = 'green' if true_label == pred_label else 'red'
    ax.set_title(f"真实: {class_names[true_label]}\n预测: {class_names[pred_label]}",
                color=color, fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

# 10. 简化的分类报告
print("\n各类别准确率:")
for i in range(10):
    class_mask = np.array(all_labels) == i
    if np.sum(class_mask) > 0:
        class_acc = np.mean(np.array(all_preds)[class_mask] == i) * 100
        print(f"{class_names[i]}: {class_acc:.1f}%")

print(f"\n总体准确率: {test_accuracy:.2f}%")
print("训练完成！这是一个轻量级版本，适合快速测试和学习。")

开始加载CIFAR-10数据集...
Files already downloaded and verified
Files already downloaded and verified
数据集下载完成！
训练样本数: 1000
测试样本数: 200
使用设备: cuda
模型参数数量: 136,874
开始快速训练...


RuntimeError: Numpy is not available